In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import json
import nltk
import re
from nltk.corpus import stopwords
from langdetect import detect
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# Uploading Dataset and inspecting its contents
df = pd.DataFrame()
df = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Video_Games_Sales_as_at_22_Dec_2016.csv', index_col=0)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Dropped all missing values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6825 entries, Wii Sports to STORM: Frontline Nation
Data columns (total 15 columns):
Platform           6825 non-null object
Year_of_Release    6825 non-null float64
Genre              6825 non-null object
Publisher          6825 non-null object
NA_Sales           6825 non-null float64
EU_Sales           6825 non-null float64
JP_Sales           6825 non-null float64
Other_Sales        6825 non-null float64
Global_Sales       6825 non-null float64
Critic_Score       6825 non-null float64
Critic_Count       6825 non-null float64
User_Score         6825 non-null object
User_Count         6825 non-null float64
Developer          6825 non-null object
Rating             6825 non-null object
dtypes: float64(9), object(6)
memory usage: 853.1+ KB


In [4]:
# Changed Release Year to integers
df['Year_of_Release'] = (df['Year_of_Release']).astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [5]:
# Seeing how many types of genres there are
df['Genre'].value_counts()

Action          1630
Sports           943
Shooter          864
Role-Playing     712
Racing           581
Platform         403
Misc             384
Fighting         378
Simulation       297
Strategy         267
Adventure        248
Puzzle           118
Name: Genre, dtype: int64

In [6]:
# Making columns for each type of game
df['Action'] = (df['Genre'] == 'Action').astype(int)
df['Adventure'] = (df['Genre'] == 'Adventure').astype(int)
df['Fighter'] = (df['Genre'] == 'Fighting').astype(int)
df['Platformer'] = (df['Genre'] == 'Platform').astype(int)
df['Puzzler'] = (df['Genre'] == 'Puzzle').astype(int)
df['Racing'] = (df['Genre'] == 'Racing').astype(int)
df['RPG'] = (df['Genre'] == 'Role-Playing').astype(int)
df['Shooter'] = (df['Genre'] == 'Shooter').astype(int)
df['Simulation'] = (df['Genre'] == 'Simulation').astype(int)
df['Sports'] = (df['Genre'] == 'Sports').astype(int)
df['Strategy'] = (df['Genre'] == 'Strategy').astype(int)
df['Variety'] = (df['Genre'] == 'Misc').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,Fighter,Platformer,Puzzler,Racing,RPG,Shooter,Simulation,Sports,Strategy,Variety
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,1,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,1,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,1,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,1,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# Seeing how many types of consoles there are
df['Platform'].value_counts()

PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: Platform, dtype: int64

In [8]:
# Making columns for some of the consoles
df['Game Boy Advance'] = (df['Platform'] == 'GBA').astype(int)
df['Nintendo 3DS'] = (df['Platform'] == '3DS').astype(int)
df['Nintendo DS'] = (df['Platform'] == 'DS').astype(int)
df['Nintendo GameCube'] = (df['Platform'] == 'GC').astype(int)
df['Nintendo Wii'] = (df['Platform'] == 'Wii').astype(int)
df['Nintendo Wii U'] = (df['Platform'] == 'WiiU').astype(int)
df['PC'] = (df['Platform'] == 'PC').astype(int)
df['PlayStation'] = (df['Platform'] == 'PS').astype(int)
df['PlayStation 2'] = (df['Platform'] == 'PS2').astype(int)
df['PlayStation 3'] = (df['Platform'] == 'PS3').astype(int)
df['PlayStation 4'] = (df['Platform'] == 'PS4').astype(int)
df['PlayStation Vita'] = (df['Platform'] == 'PSV').astype(int)
df['PSP'] = (df['Platform'] == 'PSP').astype(int)
df['Sega Dreamcast'] = (df['Platform'] == 'DC').astype(int)
df['Xbox'] = (df['Platform'] == 'XB').astype(int)
df['Xbox 360'] = (df['Platform'] == 'X360').astype(int)
df['Xbox One'] = (df['Platform'] == 'XOne').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,PlayStation,PlayStation 2,PlayStation 3,PlayStation 4,PlayStation Vita,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,0,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,0,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,0,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,0,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Saving DataFame to CSV
df.to_csv('vg_df.csv', sep='\t')

In [10]:
# Uploading second Dataset and inspecting its contents
df_2 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/metacritic_game_user_comments.csv', index_col=0)
df_2.head()

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA


In [11]:
# Dropped all missing values
df_2 = df_2.dropna()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283957 entries, 0 to 283982
Data columns (total 5 columns):
Title        283957 non-null object
Platform     283957 non-null object
Userscore    283957 non-null int64
Comment      283957 non-null object
Username     283957 non-null object
dtypes: int64(1), object(4)
memory usage: 13.0+ MB


In [12]:
# Checking the names of the platforms for merging later on
df_2['Platform'].value_counts()

PC                 118923
Xbox360             37413
PlayStation4        33547
PlayStation3        32426
PlayStation2        10636
XboxOne              9137
Switch               5922
Wii                  5232
3DS                  4812
Xbox                 4713
WiiU                 4419
GameCube             3479
DS                   3047
Nintendo64           2401
PlayStation          2396
PSP                  1732
GameBoyAdvance       1588
PlayStationVita      1555
Dreamcast             456
not specified         123
Name: Platform, dtype: int64

In [13]:
# Removing unncessary distinction and renaming certain platforms
df_2 = df_2.loc[(df_2['Platform'] != 'not specified'), :]
df_2['Platform'] = df_2['Platform'].replace('Xbox360', 'X360')
df_2['Platform'] = df_2['Platform'].replace('PlayStation4', 'PS4')
df_2['Platform'] = df_2['Platform'].replace('PlayStation3', 'PS3')
df_2['Platform'] = df_2['Platform'].replace('PlayStation2', 'PS2')
df_2['Platform'] = df_2['Platform'].replace('XboxOne', 'XOne')
df_2['Platform'] = df_2['Platform'].replace('Xbox', 'XB')
df_2['Platform'] = df_2['Platform'].replace('GameCube', 'GC')
df_2['Platform'] = df_2['Platform'].replace('PlayStation', 'PS')
df_2['Platform'] = df_2['Platform'].replace('GameBoyAdvance', 'GBA')
df_2['Platform'] = df_2['Platform'].replace('PlayStationVita', 'PSV')
df_2['Platform'] = df_2['Platform'].replace('Dreamcast', 'DC')

In [14]:
# Apply this function to clean up data, get rid of non-letters
def remove_char(x):
    """Removes non-letters and extra white space"""
    x = re.sub(r'[^a-zA-Z ]', '', x)
    x = re.sub(r'\s+', ' ', x)
    return x

df_2['Comment'] = df_2.loc[:, 'Comment'].apply(lambda x: remove_char(x))

In [15]:
# Keeping all rows that have features
df_2 = df_2.loc[(df_2['Comment'] != ' '), :]
df_2 = df_2.loc[(df_2['Comment'] != ''), :]
df_2.head()

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Everything in OoT is so near at perfection its...,SirCaestus
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I wont bore you with what everyone is already ...,Kaistlin
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a or an...,Jacody
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Im one of those people who think that this is ...,doodlerman
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA


In [16]:
# Make sure the review is in English
df_2['Language'] = df_2.loc[:, 'Comment'].apply(lambda x: detect(x.decode('utf8')))
df_2 = df_2.loc[df_2['Language'] == 'en']

In [17]:
# Merging Reviews with earlier DataFrame, resetting its index to join on name of games
df = df.reset_index()
df_w = pd.merge(df, df_2, left_on=['Name', 'Platform'], right_on=['Title', 'Platform'])
df_w.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One,Title,Userscore,Comment,Username,Language
0,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,Mario Kart Wii,7,Mario Kart Wii is one of the best racing games...,GamerOfGames,en
1,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,Mario Kart Wii,7,This is definitely the pinnacle of the Mario K...,MentalVortex,en
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,Mario Kart Wii,10,Edge magazine is a joke When a game scores co...,ErkH.,en
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,Mario Kart Wii,10,Addicting and fun Super frustrating but in a g...,Carlsdil000,en
4,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,Mario Kart Wii,10,Mario Kart wii is truly an amazing game Good s...,SonicDude,en


In [18]:
# No NaNs in this DataFrame
df_w.isnull().values.any()

False

In [19]:
# Removing duplicate or unnecessary columns, renaming others for clarity
df_w = df_w.drop(['Title'], axis=1)
df_w = df_w.drop(['Language'], axis=1)
df_w.rename(columns={'User_Score':'Average_User_Score'}, inplace=True)
df_w.rename(columns={'Userscore':'Individual_User_Score'}, inplace=True)
df_w.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,PlayStation 4,PlayStation Vita,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One,Individual_User_Score,Comment,Username
0,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,0,0,7,Mario Kart Wii is one of the best racing games...,GamerOfGames
1,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,0,0,7,This is definitely the pinnacle of the Mario K...,MentalVortex
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,0,0,10,Edge magazine is a joke When a game scores co...,ErkH.
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,0,0,10,Addicting and fun Super frustrating but in a g...,Carlsdil000
4,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,...,0,0,0,0,0,0,0,10,Mario Kart wii is truly an amazing game Good s...,SonicDude


** Because of how many reviews and words there are, I must limit how many will be used otherwise the matrix will be too large and my computer will crash. To do that, I will sample 10,000 reviews and use the same process of tokenization that I will use with all reviews but keep only the tokens that have an overall mean higher than 0.001.**

In [20]:
# Sampling 10,000 reviews
df_s = df_w.sample(10000)
df_s.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,PlayStation 4,PlayStation Vita,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One,Individual_User_Score,Comment,Username
168180,Rise of the Tomb Raider,PC,2016,Adventure,Square Enix,0.00,0.11,0.00,0.01,0.12,...,0,0,0,0,0,0,0,9,Visually this game is stunning on PC no doubt ...,Scoobydooby
53868,Metal Gear Solid 3: Snake Eater,PS2,2004,Action,Konami Digital Entertainment,1.46,0.00,0.83,1.93,4.23,...,0,0,0,0,0,0,0,10,metal gear solid is the best one in the series...,Basem101
57420,Far Cry 4,PS4,2014,Shooter,Ubisoft,1.13,2.18,0.10,0.63,4.04,...,1,0,0,0,0,0,0,7,it is the same that the rest of Far Cry I have...,davekinf
136027,TimeSplitters 2,PS2,2002,Shooter,Eidos Interactive,0.34,0.27,0.00,0.09,0.70,...,0,0,0,0,0,0,0,10,TimeSplitters is one of the best shooters ever...,shredhead
122400,Borderlands 2,PC,2012,Shooter,Take-Two Interactive,0.42,0.41,0.00,0.10,0.93,...,0,0,0,0,0,0,0,7,Borderlands has the same plusses and minuses a...,Hemiovoid


In [21]:
# Function to tokenize words from sample Video Game Reviews
def sent_tokenizer(x):
    """Tokenizes words, removes stops words, breaks down the stems and counts the tokens"""
    x = x.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    w_tokens = tokenizer.tokenize(x)
    sr = stopwords.words('english')
    for token in w_tokens:
        if token in sr:
            w_tokens.remove(token)
    ps = PorterStemmer()
    w_tokens2 = [ps.stem(w) for w in w_tokens]
    w_count = Counter(w_tokens2)
    return w_count, w_tokens2

# Creating a dictionary of dictionaries with words and their counts
tokens = df_s['Comment'].apply(lambda x: sent_tokenizer(x)[0])
tokens

168180    {u'all': 2, u'gamebeen': 2, u'go': 2, u'polish...
53868     {u'ive': 2, u'love': 1, u'is': 2, u'continu': ...
57420     {u'enjoy': 1, u'the': 1, u'play': 1, u'mountai...
136027    {u'pleas': 2, u'control': 2, u'all': 2, u'just...
122400    {u'origin': 1, u'dont': 1, u'feel': 2, u'it': ...
37985     {u'cute': 2, u'all': 2, u'show': 2, u'go': 1, ...
87156     {u'often': 2, u'cabl': 1, u'terribl': 2, u'is'...
100159    {u'all': 1, u'brath': 1, u'dont': 1, u'one': 1...
174634    {u'set': 2, u'rocket': 2, u'ai': 1, u'qualiti'...
20939     {u'ad': 2, u'get': 2, u'top': 2, u'ea': 2, u'b...
61792     {u'enjoy': 1, u'is': 1, u'am': 1, u'player': 1...
142782    {u'just': 1, u'less': 1, u'money': 1, u'lack':...
115066    {u'all': 3, u'unchart': 1, u'asylum': 1, u'ver...
175785    {u'enjoy': 1, u'prevent': 1, u'consid': 1, u'c...
57409     {u'set': 2, u'point': 2, u'is': 3, u'lack': 2,...
1564      {u'load': 2, u'control': 1, u'chore': 2, u'pla...
139220    {u'metallica': 1, u'forget': 2

In [22]:
# Making a list of the dictionary
tokens = list(tokens)

In [23]:
# Creating a DataFrame of the list
df_t = pd.DataFrame(tokens)
df_t.head()

,a,aa,aaa,aaaaaaaaaa,aaaaaam,aaaahhhh,aaaccess,aaaproduct,aaatitl,aacon,...,zon,zone,zonebottom,zoneit,zonesuperfici,zooland,zoom,zooooooooooooomgggggggg,zotac,zweihand
0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Naming the index, changing missing values to 0
df_t.index.name = 'Reviews'
df_t = df_t.replace(pd.np.nan, 0)
df_t.head()

,a,aa,aaa,aaaaaaaaaa,aaaaaam,aaaahhhh,aaaccess,aaaproduct,aaatitl,aacon,...,zon,zone,zonebottom,zoneit,zonesuperfici,zooland,zoom,zooooooooooooomgggggggg,zotac,zweihand
Reviews,,,,,,,,,,,,,,,,,,,,,
0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Putting DataFrame through a TF-IDF Transformer
transformer = TfidfTransformer()
matrix = transformer.fit_transform(df_t)
matrix = matrix.todense()
df_wt = pd.DataFrame(matrix, columns=df_t.columns)
df_wt.index.name = 'Reviews'
df_wt.head()

,a,aa,aaa,aaaaaaaaaa,aaaaaam,aaaahhhh,aaaccess,aaaproduct,aaatitl,aacon,...,zon,zone,zonebottom,zoneit,zonesuperfici,zooland,zoom,zooooooooooooomgggggggg,zotac,zweihand
Reviews,,,,,,,,,,,,,,,,,,,,,
0,0.081952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.023941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Filtering by mean greater than 0.001, removes unlikely words
df_wt_mean = df_wt.mean()
tokens_to_keep = df_wt_mean[df_wt_mean  > 0.001].keys()
tokens_to_keep

Index([      u'a',    u'abil',     u'abl',   u'about', u'absolut',      u'ac',
       u'account',  u'achiev',  u'across',     u'act',
       ...
          u'yeah',    u'year',     u'yet',     u'you',   u'youll',    u'your',
          u'youv',   u'zelda',    u'zero',   u'zombi'],
      dtype='object', length=1185)

** Now that I have the more important words from the random sample, I will run the same tokenization process with the rest of the data, the only difference being that it will only keep words that are proven to be important.**

In [27]:
# Function to tokenize words from Video Game Reviews
def sent_tokenizer_2(x, words):
    """Tokenizes words, removes stops words, breaks down the stems, counts the tokens and 
    makes sure they are part of the main set of tokens"""
    x = x.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    w_tokens = tokenizer.tokenize(x)
    sr = stopwords.words('english')
    for token in w_tokens:
        if token in sr:
            w_tokens.remove(token)
    ps = PorterStemmer()
    w_tokens2 = [ps.stem(w) for w in w_tokens if w in words]
    w_count = Counter(w_tokens2)
    return w_count, w_tokens2

# Creating a dictionary of dictionaries with words and their counts
tokens_2 = df_w['Comment'].apply(lambda x: sent_tokenizer_2(x, tokens_to_keep)[0])
tokens_2

0         {u'still': 1, u'dont': 3, u'kart': 5, u'feel':...
1         {u'enjoy': 1, u'gone': 1, u'often': 1, u'kart'...
2         {u'a': 3, u'never': 1, u'there': 1, u'kart': 1...
3         {u'good': 1, u'often': 1, u'would': 1, u'there...
4         {u'great': 1, u'good': 2, u'kart': 1, u'wii': ...
5         {u'even': 1, u'great': 1, u'good': 1, u'get': ...
6                       {u'fun': 1, u'game': 1, u'play': 1}
7         {u'ive': 2, u'all': 1, u'dont': 1, u'kart': 4,...
8         {u'a': 1, u'good': 1, u'old': 1, u'like': 1, u...
9         {u'just': 3, u'four': 3, u'isnt': 3, u'still':...
10        {u'real': 1, u'a': 1, u'good': 1, u'isnt': 1, ...
11        {u'control': 1, u'all': 1, u'kart': 2, u'ai': ...
12        {u'just': 2, u'is': 3, u'wii': 6, u'player': 3...
13        {u'a': 1, u'gone': 1, u'all': 1, u'good': 1, u...
14        {u'dont': 2, u'do': 2, u'some': 2, u'are': 2, ...
15        {u'old': 1, u'like': 1, u'ten': 1, u'extra': 1...
16        {u'a': 1, u'play': 1, u'love':

In [28]:
# Making a list of the dictionary
tokens_2 = list(tokens_2)

In [29]:
# Creating a DataFrame of the list
df_t_2 = pd.DataFrame(tokens_2)
df_t_2.head()

,a,about,absolut,ac,account,across,act,action,activ,actual,...,yeah,year,yet,you,youll,your,youv,zelda,zero,zombi
0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Naming the index, changing missing values to 0
df_t_2.index.name = 'Reviews'
df_t_2 = df_t_2.replace(pd.np.nan, 0)
df_t_2.head()

,a,about,absolut,ac,account,across,act,action,activ,actual,...,yeah,year,yet,you,youll,your,youv,zelda,zero,zombi
Reviews,,,,,,,,,,,,,,,,,,,,,
0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Putting DataFrame through a TF-IDF Transformer
transformer = TfidfTransformer()
matrix = transformer.fit_transform(df_t_2)
matrix = matrix.todense()
df_wt_2 = pd.DataFrame(matrix, columns=df_t_2.columns)
df_wt_2.index.name = 'Reviews'
df_wt_2.head()

,a,about,absolut,ac,account,across,act,action,activ,actual,...,yeah,year,yet,you,youll,your,youv,zelda,zero,zombi
Reviews,,,,,,,,,,,,,,,,,,,,,
0,0.129242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.059922,0.0,0.044656,0.0,0.0,0.0,0.0
1,0.034115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.250118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [32]:
# Resetting indices of df_w and df_wt_2 for merging, keeping only necessary columns for NLP
df_w = df_w.reset_index()
df_wt_2 = df_wt_2.reset_index()
df_nlp = pd.merge(df_w[['index', 'Name', 'Platform']], df_wt_2, left_on='index', right_on='Reviews')
df_nlp.head()

,index,Name,Platform,Reviews,a,about,absolut,ac,account,across,...,yeah,year,yet,you,youll,your,youv,zelda,zero,zombi
0,0,Mario Kart Wii,Wii,0,0.129242,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.059922,0.0,0.044656,0.0,0.0,0.0,0.0
1,1,Mario Kart Wii,Wii,1,0.034115,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,2,Mario Kart Wii,Wii,2,0.250118,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,3,Mario Kart Wii,Wii,3,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,4,Mario Kart Wii,Wii,4,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [33]:
# Dropping Index and Reviews for Groupby, getting mean of each word for each game
df_nlp = df_nlp.drop(['index'], axis=1)
df_nlp = df_nlp.drop(['Reviews'], axis=1)
df_nlp = df_nlp.groupby(['Name', 'Platform']).mean()
df_nlp.head()

a     about  absolut   ac  \
Name                             Platform                                     
2010 FIFA World Cup South Africa PS3       0.077199  0.000000      0.0  0.0   
                                 X360      0.036675  0.000000      0.0  0.0   
ATV Offroad Fury                 PS2       0.034426  0.000000      0.0  0.0   
ATV Offroad Fury 2               PS2       0.035777  0.000000      0.0  0.0   
Ace Combat 04: Shattered Skies   PS2       0.053300  0.001371      0.0  0.0   

                                           account  across  act    action  \
Name                             Platform                                   
2010 FIFA World Cup South Africa PS3           0.0     0.0  0.0  0.000000   
                                 X360          0.0     0.0  0.0  0.000000   
ATV Offroad Fury                 PS2           0.0     0.0  0.0  0.039822   
ATV Offroad Fury 2               PS2           0.0     0.0  0.0  0.000000   
Ace Combat 04: Shattered Skies   PS2           0.0     0.0  0.0  0.000000   

                                           activ    actual  ...    yeah  \
Name                             Platform                   ...           
2010 FIFA World Cup South Africa PS3         0.0  0.000000  ...     0.0   
                                 X360        0.0  0.000000  ...     0.0   
ATV Offroad Fury                 PS2         0.0  0.000000  ...     0.0   
ATV Offroad Fury 2               PS2         0.0  0.000000  ...     0.0   
Ace Combat 04: Shattered Skies   PS2         0.0  0.001677  ...     0.0   

                                               year       yet       you  \
Name                             Platform                                 
2010 FIFA World Cup South Africa PS3       0.000000  0.000000  0.064435   
                                 X360      0.042606  0.000000  0.022417   
ATV Offroad Fury                 PS2       0.000000  0.012096  0.020864   
ATV Offroad Fury 2               PS2       0.000000  0.000000  0.067344   
Ace Combat 04: Shattered Skies   PS2       0.007696  0.010040  0.030543   

                                              youll      your  youv  zelda  \
Name                             Platform                                    
2010 FIFA World Cup South Africa PS3       0.000000  0.000000   0.0    0.0   
                                 X360      0.000000  0.000000   0.0    0.0   
ATV Offroad Fury                 PS2       0.000000  0.000000   0.0    0.0   
ATV Offroad Fury 2               PS2       0.000000  0.000000   0.0    0.0   
Ace Combat 04: Shattered Skies   PS2       0.005365  0.003983   0.0    0.0   

                                               zero  zombi  
Name                             Platform                   
2010 FIFA World Cup South Africa PS3       0.000000    0.0  
                                 X360      0.000000    0.0  
ATV Offroad Fury                 PS2       0.004288    0.0  
ATV Offroad Fury 2               PS2       0.000000    0.0  
Ace Combat 04: Shattered Skies   PS2       0.000000    0.0  

[5 rows x 988 columns]

In [34]:
# Dropping Duplicates of df_w and unnecessary columns, resetting index of df_nlp for merging
df_w = df_w.drop_duplicates(['Name', 'Platform'])
df_w = df_w.drop(['Individual_User_Score'], axis=1)
df_w = df_w.drop(['Comment'], axis=1)
df_w = df_w.drop(['Username'], axis=1)
df_nlp = df_nlp.reset_index()

In [35]:
# Merging df_nlp and df_w to have all the features needed for Machine Learning
df_nlp = pd.merge(df_nlp, df_w, left_on=['Name', 'Platform'], right_on=['Name', 'Platform'])
df_nlp.head()

,Name,Platform,a,about,absolut,ac,account,across,act,action,...,PlayStation,PlayStation 2,PlayStation 3,PlayStation 4,PlayStation Vita,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One
0,2010 FIFA World Cup South Africa,PS3,0.077199,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,0,0,0
1,2010 FIFA World Cup South Africa,X360,0.036675,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,0,0,0,0,0,0,0,1,0
2,ATV Offroad Fury,PS2,0.034426,0.000000,0.0,0.0,0.0,0.0,0.0,0.039822,...,0,1,0,0,0,0,0,0,0,0
3,ATV Offroad Fury 2,PS2,0.035777,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,1,0,0,0,0,0,0,0,0
4,Ace Combat 04: Shattered Skies,PS2,0.053300,0.001371,0.0,0.0,0.0,0.0,0.0,0.000000,...,0,1,0,0,0,0,0,0,0,0


In [36]:
# Saving DataFame to CSV if needed
df_nlp.to_csv('vg_nlp_df.csv', sep='\t')

In [37]:
# Uploading third Dataset and inspecting its contents
df_3 = pd.DataFrame()
df_3 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Managerial_and_Decision_Economics_2013_Video_Games_Dataset.csv', index_col=0)
df_3.head()

,Title,US Sales (millions),Block4,Block2,Block1,Block0.5,YearReleased,2004,2005,2006,...,Tricks,Volleyball,Wakeboarding,Wrestling,FirstPerson,Platform,Isometric,SideScrolling,TopDown,ThirdPerson
Console,,,,,,,,,,,,,,,,,,,,,
Nintendo DS,Super Mario 64 DS,4.69,1,1,1,1,2004,1,0,0,...,0,0,0,0,0,1,0,0,0,1
Sony PSP,Lumines: Puzzle Fusion,0.56,0,0,0,1,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Nintendo DS,WarioWare Touched!,0.54,0,0,0,1,2004,1,0,0,...,0,0,0,0,0,1,1,1,1,1
Sony PSP,Hot Shots Golf: Open Tee,0.49,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,0,0,0,0,1
Nintendo DS,Spider-Man 2,0.45,0,0,0,0,2004,1,0,0,...,0,0,0,0,0,1,0,1,0,1


In [38]:
# No NaNs in this DataFrame
df_3.isnull().values.any()

False

In [39]:
# Saving DataFame to CSV
df_3.to_csv('vg_2_df.csv', sep='\t')